Problem generator

In [11]:
N_VALUES = [5, 10, 20, 100, 500, 1000]

import random

def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

Greedy solution

In [12]:
import logging


def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    while goal != covered:
        x = all_lists.pop(0)
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")

Greedy solution log

In [13]:
logging.getLogger().setLevel(logging.INFO)
for N in N_VALUES:
    greedy(N)

INFO:root:Greedy solution for N=5: w=5 (bloat=0%)
INFO:root:Greedy solution for N=10: w=13 (bloat=30%)
INFO:root:Greedy solution for N=20: w=46 (bloat=130%)
INFO:root:Greedy solution for N=100: w=332 (bloat=232%)
INFO:root:Greedy solution for N=500: w=2162 (bloat=332%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)


Proposed solution

In [14]:
def proposed_solution(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    first_iter = True
    while goal != covered:
        if first_iter:
            x = x = all_lists.pop()
            first_iter = False
        else:
            x = all_lists.pop(0)
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
            f"Proposed solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)")


logging.getLogger().setLevel(logging.INFO)
for N in N_VALUES:
    proposed_solution(N)

INFO:root:Proposed solution for N=5: w=5 (bloat=0%)
INFO:root:Proposed solution for N=10: w=14 (bloat=40%)
INFO:root:Proposed solution for N=20: w=36 (bloat=80%)
INFO:root:Proposed solution for N=100: w=340 (bloat=240%)
INFO:root:Proposed solution for N=500: w=2187 (bloat=337%)
INFO:root:Proposed solution for N=1000: w=4699 (bloat=370%)
